In [15]:
import pandas as pd
from transformers import AutoTokenizer

model_name = "Qwen/Qwen3-0.6B"

In [16]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

### epoch 1, N_sup = 8

In [7]:
df1 = pd.read_json('evals/checkpoint_epoch_1_Nsup8.jsonl', lines=True)

In [9]:
df1

,id,prompt,generated_answer,final_answer,is_match
0,1,<|endoftext|><|endoftext|><|endoftext|><|endof...,<think>\nThe total number of eggs she eats in ...,16,False
1,2,<|endoftext|><|endoftext|><|endoftext|><|endof...,<think>\nThe total number of bolts of fabric i...,<think>\nThe total number of bolts of fabric i...,False
2,3,<|endoftext|><|endoftext|><|endoftext|><|endof...,"<think>\nThe total cost of the house is $80,00...","65,000",False
3,4,<|endoftext|><|endoftext|><|endoftext|><|endof...,<think>\nThe total number of meters he runs in...,180,False
4,5,"<|im_start|>user\nEvery day, Wendi feeds each ...",<think>\nThe total number of cups of feed that...,30,False
...,...,...,...,...,...
195,196,<|endoftext|><|endoftext|><|endoftext|><|endof...,<think>\nThe cost of a pen is $1.20 + $0.30 = ...,24.00,False
196,197,<|endoftext|><|endoftext|><|endoftext|><|endof...,<think>\nThe total amount of carrots is 200 po...,120,True
197,198,<|endoftext|><|endoftext|><|endoftext|><|endof...,<think>\nThe total number of people in the cit...,16276,False
198,199,<|endoftext|><|endoftext|><|endoftext|><|endof...,<think>\nThe total number of cards she has is ...,140,False


In [11]:
df1['is_match'].mean()

np.float64(0.195)

In [29]:
print(df1.iloc[7]['prompt'])

<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|im_start|>user
Carla is downloading a 200 GB file. Normally she can download 2 GB/minute, but 40% of the way through the download, Windows forces a restart to install updates, which takes 20 minutes. Then Carla has to restart the download from the beginning. How load does it take to download the file?<|im_end|>
<|im_start|>assistant



In [28]:
print(df1['generated_answer'].iloc[7])

<think>
The total time is 200/2 = <<200/2=100>>100 minutes.
The first 100 minutes is 100 minutes.
The second 20 minutes is 20 minutes.
So the total time is 100+20 = <<120>>120 minutes.
</think>

Answer: 120 minutes<|im_end|>
<|endoftext|>The total time to download the 200-GB file is 120 minutes.<|im_end|>
<|endoftext|>In a 200-GB file, Carla can download 200/2 = <<200/2=100>>100 minutes on the first 100 minutes of the file. 40% of the time is 20 minutes of the second 20 minutes of the 200-GB file. 100+20 = <<120>>120 minutes in total.<|im_end|>
<|endoftext|>In a 200-GB file, the total time to be downloaded is 120 minutes.<|im_end|>
<|endoftext|>Answer: 120<|im_end|>
<|endoftext|>Answer: 120<|im_end|>
<|endoftext|>


In [22]:
df1['generated_answer'].apply(lambda ga: ga.split(tokenizer.special_tokens_map['eos_token'])[0].strip()).iloc[7]

'<think>\nThe total time is 200/2 = <<200/2=100>>100 minutes.\nThe first 100 minutes is 100 minutes.\nThe second 20 minutes is 20 minutes.\nSo the total time is 100+20 = <<120>>120 minutes.\n</think>\n\nAnswer: 120 minutes'

In [25]:
def process_answer(ga):
    # Cut off parts after EOS token
    ga = ga.split(tokenizer.special_tokens_map['eos_token'])[0].strip()

    # Extract the final answer after the last "Answer:" marker
    if "Answer:" in ga:
        ga = ga.split("Answer:")[-1].strip()
    return ga

df1['new_final_answer'] = df1['generated_answer'].apply(process_answer)

In [26]:
df1.iloc[7][['final_answer', 'new_final_answer']]

final_answer                120
new_final_answer    120 minutes
Name: 7, dtype: object

In [ ]:
eval_loader = get_generation_dataloader(
    tokenizer, args.max_length, args.eval_batch_size,
    seed=args.seed, train=False, num_samples=args.num_eval_samples
)

### epoch 1, N_sup = 1

In [13]:
df2 = pd.read_json('evals/checkpoint_epoch_1_Nsup1.jsonl', lines=True)

In [14]:
df2['is_match'].mean()

np.float64(0.2)